In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/cs-training_ver02.csv')
x = df.drop(columns='seriousdlqin2yrs')
y = df.seriousdlqin2yrs

In [3]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
pos_weight = y.value_counts()[0] / y.value_counts()[1]
xgb = XGBClassifier(objective='binary:logistic', scale_pos_weight=pos_weight)
param = {'n_estimators': np.arange(2, 301),
         'learning_rate' : np.arange(1,51)/100,
         'max_depth':np.arange(2,11)}
xgb_best = RandomizedSearchCV(estimator=xgb, param_distributions=param,
                             n_iter=20, cv=4, scoring='roc_auc', n_jobs=-1,
                             verbose=1)
xgb_best.fit(x, y)

Fitting 4 folds for each of 20 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  1.6min finished


RandomizedSearchCV(cv=4,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
       210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222,
       223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235,
       236, 237, 238, 239, 240, 241, 

In [4]:
import joblib
from sklearn.model_selection import cross_val_score

In [5]:
with open('baseline.pkl', 'rb') as f:
    data = joblib.load(f)
data.keys()
best_baseline = data['best_baseline']

In [6]:
print(cross_val_score(best_baseline, x, y, cv=4, scoring='roc_auc').mean())
print(cross_val_score(xgb_best.best_estimator_, x, y, cv=4, scoring='roc_auc').mean())

0.8487194166095416
0.8642567979154865
